In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from string import punctuation
punctuation = punctuation + '’'

In [3]:
words_synonyms = {
'abash': 'disconcert',
'abate': 'alleviate',
'abbreviate': 'compress',
'abhorrent': 'offensive',
'abject': 'despicable',
'abjure': 'forsake',
'abortive': 'vain',
'abound': 'flourish',
'abrupt': 'hasty',
'absolute': 'genuine',
'absolve': 'pardon',
'abundant': 'lavish',
'access': 'obtain',
'acclaimed': 'celebrated',
'accord': 'agreement',
'accountable': 'liable',
'accurate': 'true',
'accusation': 'invective',
'acquire': 'have',
'acrimony': 'harshness',
'active': 'busy',
'acumen': 'reason',
'acute': 'cunning',
'adage': 'axiom',
'adamant': 'stubborn',
'adherent': 'follower',
'adjunct': 'joined',
'admonish': 'counsel',
'adoration': 'honor',
'adventurous': 'bold',
'adversity': 'calamity',
'aged': 'old',
'agree': 'consent',
'agreement': 'concord',
'aid': 'help',
'alien': 'foreigner',
'allay': 'pacify',
'alleviate': 'mitigate',
'allude': 'insinuate',
'allure': 'entice',
'alluring': 'bewitching',
'altruism': 'generosity',
'amazing': 'incredible',
'amend': 'rectify',
'amicable': 'tranquil',
'amplify': 'augment',
'anger': 'enrage',
'angry': 'wrathful',
'annul': 'rescind',
'answer': 'reply',
'apartment': 'tenement',
'appointed': 'assigned',
'appreciate': 'enjoy',
'approach': 'come',
'approve': 'concur',
'arcane': 'obscure',
'arraign': 'incriminate',
'ascend': 'climb',
'ask': 'question',
'assembly': 'throng',
'assertive': 'trenchant',
'assigned': 'appointed',
'astound': 'baffle',
'attract': 'lure',
'attribute': 'impute',
'audacity': 'boldness',
'authority': 'power',
'augment': 'amplify',
'austere': 'bleak',
'authentic': 'accurate',
'awareness': 'acumen',
'awful': 'dreadful',
'awkward': 'uncouth',
'axiom': 'adage',
'bad': 'depraved',
'baffle': 'astound',
'baneful': 'noxious',
'barbarous': 'frustrate',
'baroque': 'florid',
'barren': 'sterile',
'barricade': 'barrier',
'barrier': 'barricade',
'base': 'vulgar',
'batty': 'insane',
'bawdy': 'coarse',
'beautiful': 'winsome',
'becloud': 'befogged',
'befogged': 'becloud',
'begin': 'start',
'begun': 'started',
'beginner': 'tyro',
'beginning': 'inception',
'beguile': 'captivate',
'benevolent': 'benign',
'benign': 'benevolent',
'benefiting': 'enjoying',
'bent': 'crooked',
'bewitching': 'alluring',
'big': 'enormous',
'bind': 'predicament',
'blame': 'denounce',
'bleak': 'gloomy',
'blemish': 'stain',
'blunt': 'candid',
'boisterous': 'clamorous',
'bold': 'confident',
'boldness': 'audacity',
'brave': 'courageous',
'bravery': 'valor',
'breakable': 'brittle',
'bright': 'sparkling',
'brittle': 'breakable',
'bunched': 'compact',
'bustle': 'commotion',
'busy': 'active',
'cajole': 'persuade',
'calamity': 'adversity',
'calculating': 'canny',
'callous': 'insensitive',
'calm': 'harmonious',
'calumny': 'defamation',
'camouflage': 'cloak',
'cancel': 'nullify',
'candid': 'blunt',
'canny': 'calculating',
'capable': 'competent',
'captivate': 'beguile',
'captivity': 'imprisonment',
'carnal': 'earthly',
'castigate': 'condemn',
'catastrophic': 'destructive',
'catholic': 'generic',
'cautious': 'wary',
'cease': 'stop',
'celebrated': 'acclaimed',
'cement': 'plaster',
'censure': 'remonstrate',
'centre': 'core',
'centralize': 'consolidate',
'certain': 'definite',
'changed': 'evolved',
'chagrin': 'umbrage',
'chaste': 'virtuous',
'chastise': 'punish',
'cheap': 'competitive',
'cherish': 'sacred',
'chilly': 'cool',
'choice': 'option',
'choices': 'options',
'chunky': 'thick',
'clamorous': 'boisterous',
'clandestine': 'covert',
'classic': 'simple',
'climb': 'ascend',
'cloak': 'camouflage',
'clown': 'comic',
'coarse': 'bawdy',
'collapse': 'decay',
'core': 'middle',
'come': 'approach',
'comic': 'clown',
'commotion': 'bustle',
'compact': 'bunched',
'compassion': 'kindness',
'compassionate': 'lenient',
'competent': 'capable',
'competitive': 'cheap',
'complaisant': 'obliging',
'completely': 'utterly',
'compliant': 'tame',
'complicated': 'knotty',
'compress': 'abbreviate',
'comprise': 'include',
'conceal': 'hide',
'concede': 'yield',
'conceit': 'vanity',
'concept': 'idea',
'concise': 'succinct',
'concur': 'approve',
'condemn': 'castigate',
'confirm': 'vouch',
'conformity': 'harmony',
'connect': 'yoke',
'consent': 'ratify',
'consequence': 'effect',
'consequent': 'subsequent',
'consider': 'think',
'considerable': 'substantial',
'consolidate': 'centralize',
'conspicuous': 'prominent',
'construct': 'fabricate',
'contaminate': 'defile',
'contempt': 'scorn',
'contradict': 'deny',
'contrary': 'dissimilar',
'converge': 'gather',
'cool': 'temperate',
'correct': 'right',
'corrupt': 'vicious',
'counsel': 'admonish',
'courageous': 'brave',
'course': 'steep',
'courteous': 'gracious',
'covert': 'clandestine',
'creation': 'formation',
'cripple': 'hamstrung',
'crooked': 'bent',
'cruel': 'ferocious',
'cry': 'weep',
'cryptic': 'oracular',
'cunning': 'shrewd',
'cut': 'slice',
'damage': 'hurt',
'damned': 'infernal',
'dangerous': 'perilous',
'dark': 'shadowy',
'death knell': 'knell',
'decay': 'collapse',
'deceit': 'subterfuge',
'deception': 'hypocrisy',
'decide': 'determine',
'decipher': 'interpret',
'decline': 'wane',
'decorative': 'ornamental',
'defamation': 'calumny',
'defame': 'slander',
'defend': 'justify',
'defile': 'contaminate',
'definite': 'certain',
'deflect': 'fluctuate',
'defray': 'spend',
'deliberate': 'cautious',
'delicious': 'savory',
'delight': 'zest',
'delusion': 'lunacy',
'demolish': 'subvert',
'denounce': 'blame',
'dense': 'opaque',
'deny': 'contradict',
'depraved': 'bad',
'depression': 'despair',
'deprive': 'despoil',
'deride': 'mock',
'derogatory': 'sarcastic',
'descendant': 'offspring',
'describe': 'portray',
'design': 'make',
'desolate': 'barren',
'despair': 'depression',
'despicable': 'abject',
'despise': 'hate',
'despoil': 'deprive',
'destitute': 'indigent',
'destroy': 'ravage',
'destructive': 'catastrophic',
'detain': 'restrain',
'determine': 'decide',
'detest': 'disdain',
'diaphanous': 'transparent',
'difference': 'disagreement',
'diffident': 'timid',
'dignity': 'glory',
'diminish': 'impair',
'diminution': 'eclipse',
'diminutive': 'dwarf',
'dinky': 'little',
'dirty': 'squalid',
'disagreement': 'difference',
'disclose': 'tell',
'disconcert': 'abash',
'disdain': 'detest',
'disgusting': 'grisly',
'dishonest': 'treacherous',
'display': 'show',
'displeasure': 'resentment',
'disposition': 'inclination',
'dissimilar': 'contrary',
'distress': 'trouble',
'do': 'execute',
'docile': 'pliable',
'dominion': 'supremacy', 
'doubtful': 'precarious',
'dreadful': 'awful',
'drop': 'fall',
'dull': 'jejune',
'dusky': 'murky',
'dwarf': 'diminutive',
'eager': 'keen',
'eagerness': 'zeal',
'earthly': 'carnal',
'eccentric': 'strange',
'eclipse': 'diminution',
'economy': 'frugality',
'ecstasy': 'delight',
'educate': 'teach',
'education': 'learning',
'efface': 'destroy',
'effect': 'consequence',
'egotism': 'conceit',
'elaborate': 'explain',
'eloquence': 'expression',
'eloquent': 'vivid',
'enchanting': 'interesting',
'encumbrance': 'hindrance',
'endeavour': 'undertake',
'enjoy': 'appreciate',
'enormous': 'big',
'enrage': 'anger',
'entice': 'allure',
'entreat': 'solicit',
'epitome': 'precise',
'equitable': 'indifferent',
'equivocal': 'uncertain',
'equivocate': 'quibble',
'eruption': 'outbreak',
'esteemed': 'venerable',
'ethic': 'virtue',
'evident': 'obvious',
'evolved': 'changed',
'excellent': 'good',
'execute': 'do',
'exert': 'wield',
'explain': 'elaborate',
'expression': 'eloquence',
'exquisite': 'immaculate',
'fabricate': 'construct',
'fair': 'just',
'fake': 'false',
'fall': 'drop',
'fallacy': 'delusion',
'falter': 'stumble',
'fanatical': 'narrow-minded',
'fast': 'quick',
'fat': 'stout',
'fear': 'fright',
'feeble': 'weak',
'ferocious': 'cruel',
'feud': 'strife',
'fitness': 'wellness',
'flaky': 'impulsive',
'flatter': 'pamper',
'flippancy': 'pertness',
'florid': 'baroque',
'flourish': 'abound',
'fluctuate': 'deflect',
'flustered': 'perturbed',
'fly': 'soar',
'follower': 'adherent',
'foreigner': 'alien',
'foremost': 'paramount',
'formation': 'creation',
'forsake': 'abjure',
'foundling': 'urchin',
'fracture': 'break',
'frantic': 'violent',
'fright': 'fear',
'frightened': 'startled',
'frightful': 'hideous',
'frivolous': 'petty',
'frolicsome': 'jovial',
'frugal': 'thrifty',
'frugality': 'economy',
'frustrate': 'barbarous',
'funny': 'humorous',
'gallant': 'masculine',
'gather': 'converge',
'gathering': 'assembly',
'generic': 'catholic',
'generosity': 'courtesy',
'genuine': 'intrinsic',
'get': 'acquire',
'glimpse': 'snapshot',
'gloom': 'obscurity',
'glory': 'dignity',
'glut': 'stuff',
'go': 'recede',
'good': 'excellent',
'gorgeous': 'beautiful',
'gracious': 'courteous',
'great': 'noteworthy',
'grisly': 'disgusting',
'gross': 'improper',
'grudge': 'hatred',
'hamper': 'retard',
'hamstrung': 'cripple',
'hapless': 'unfortunate',
'happy': 'pleased',
'harass': 'molest',
'harmonious': 'calm',
'harmony': 'conformity',
'harshness': 'acrimony',
'hasten': 'hurry',
'hasty': 'abrupt',
'hate': 'despise',
'hatred': 'grudge',
'haughty': 'pompous',
'hazard': 'peril',
'health': 'medical',
'help': 'aid',
'heretic': 'non-conformist',
'hide': 'conceal',
'hideous': 'frightful',
'hindrance': 'encumbrance',
'hold': 'take',
'honor': 'adoration',
'horrible': 'ugly',
'huge': 'immense',
'humble': 'modest',
'humility': 'resignation',
'humorous': 'funny',
'hurdle': 'impediment',
'hurry': 'hasten',
'hurt': 'damage',
'hypocrisy': 'deception',
'idea': 'thought',
'idealist': 'optimist',
'immaculate': 'exquisite',
'immense': 'huge',
'immerse': 'submerge',
'immigrant': 'stranger',
'imminent': 'impending',
'immunity': 'prerogative',
'impair': 'diminish',
'impede': 'obstruct',
'impediment': 'hurdle',
'impending': 'imminent',
'impenitent': 'uncontrite',
'imperative': 'mandatory',
'impious': 'irreligious',
'important': 'necessary',
'impostor': 'quack',
'impress': 'mark',
'imprisonment': 'captivity',
'improper': 'gross',
'impudent': 'saucy',
'impulsive': 'flaky',
'impute': 'attribute',
'inappropriate': 'incongruous',
'incisive': 'terse',
'incite': 'urge',
'inclination': 'disposition',
'include': 'comprise',
'inception': 'beginning',
'incompetent': 'inefficient',
'incongruous': 'inappropriate',
'incredible': 'amazing',
'incriminate': 'arraign',
'inculcate': 'instill',
'incursion': 'raid',
'indifferent': 'nonchalant',
'indigent': 'insolvent',
'indolent': 'lazy',
'inefficient': 'incompetent',
'inevitable': 'unavoidable',
'infernal': 'damned',
'infringe': 'violate',
'ingenious': 'veteran',
'ingenuous': 'undisguised',
'insane': 'batty',
'inseminate': 'propagate',
'insensitive': 'callous',
'insinuate': 'allude',
'insipid': 'tasteless',
'instill': 'inculcate',
'institution':'organization',
'interesting': 'enchanting',
'intermittent': 'sporadic',
'interpret': 'decipher',
'intricate': 'tangled',
'intrigue': 'scheme',
'invective': 'accusation',
'invincible': 'unconquerable',
'irksome': 'monotonous',
'irreligious': 'impious',
'irrepressible': 'irresistible',
'irresistible': 'irrepressible',
'irritate': 'harass',
'jaded': 'tired',
'jejune': 'dull',
'joined': 'adjunct',
'joint': 'mutual',
'jovial': 'frolicsome',
'jubilant': 'rejoicing',
'judicious': 'thoughtful',
'justify': 'defend',
'juvenile': 'young',
'keen': 'eager',
'keep': 'hold',
'kill': 'slay',
'kindness': 'compassion',
'kindred': 'relation',
'knave': 'dishonest',
'knell': 'death knell',
'knotty': 'complicated',
'languid': 'sluggish',
'languish': 'yearn',
'latent': 'occult',
'lavish': 'abundant',
'lax': 'slack',
'lazy': 'indolent',
'learning': 'being educated',
'legislature': 'parliament',
'lenient': 'compassionate',
'lethargy': 'stupor',
'liable': 'accountable',
'liberal': 'munificent',
'linger': 'loiter',
'little': 'dinky',
'loiter': 'linger',
'look': 'scrutinize',
'love': 'like',
'lucid': 'sound',
'lure': 'attract',
'luscious': 'palatable',
'luxuriant': 'profuse',
'magnanimous': 'liberal',
'magnificent': 'sublime',
'make': 'design',
'malevolent': 'odious',
'malice': 'vengefulness',
'malign': 'vilify',
'mandatory': 'imperative',
'mark': 'impress',
'marry': 'wed',
'marvelous': 'miraculous',
'masculine': 'gallant',
'matchless': 'peerless',
'meek': 'humble',
'medical': 'health care',
'menacing': 'ominous',
'merit': 'stature',
'methods': 'ways',
'middle': 'core',
'miraculous': 'marvelous',
'mischievous': 'prankish',
'miser': 'niggardly',
'miserable': 'shabby',
'mock': 'sneer',
'mollify': 'appease',
'momentous': 'notable',
'monotonous': 'irksome',
'morbid': 'nasty',
'move': 'plod',
'moved': 'shifted',
'murky': 'dusky',
'mutinous': 'recalcitrant',
'mutual': 'joint',
'narrow-minded': 'fanatical',
'nasty': 'morbid',
'neat': 'trim',
'necessary': 'important',
'new': 'novel',
'niggardly': 'miser',
'nimble': 'spry',
'non-conformist': 'heretic',
'notable': 'momentous',
'noteworthy': 'great',
'novel': 'new',
'noxious': 'baneful',
'nullify': 'cancel',
'obliging': 'complaisant',
'oblique': 'zig-zag',
'obscurity': 'gloom',
'obstruct': 'impede',
'obtain': 'gain',
'obvious': 'evident',
'occult': 'latent',
'odious': 'malevolent',
'offence': 'outrage',
'offensive': 'abhorrent',
'offspring': 'descendant',
'old': 'aged',
'ominous': 'menacing',
'opaque': 'dense',
'optimist': 'idealist',
'option': 'choice',
'options': 'choices',
'oracular': 'cryptic',
'ordain': 'order',
'order': 'ordain',
'ornamental': 'decorative',
'outbreak': 'eruption',
'outcome': 'result',
'outcomes': 'results',
'outrage': 'offence',
'pace': 'progress',
'pacify': 'allay',
'palatable': 'luscious',
'pale': 'wan',
'pamper': 'flatter',
'panicked': 'scared',
'paramount': 'foremost',
'parasite': 'sycophant',
'pardon': 'absolve',
'parliament': 'legislature',
'part': 'portion',
'partial': 'superficial',
'peerless': 'matchless',
'peevish': 'perverse',
'people': 'individuals',
'peoples': 'individuals',
'persons': 'individuals',
'person': 'individual',
'peril': 'hazard',
'perilous': 'dangerous',
'persuade': 'cajole',
'pertness': 'flippancy',
'perturbed': 'flustered',
'perverse': 'peevish',
'petty': 'frivolous',
'place': 'put',
'plan': 'region',
'planted': 'sowed',
'plaster': 'cement',
'pleased': 'happy',
'pliable': 'docile',
'plod': 'move',
'poison': 'venom',
'pompous': 'haughty',
'portion': 'part',
'power': 'authority',
'portray': 'describe',
'prankish': 'mischievous',
'precarious': 'doubtful',
'precise': 'prompt',
'predicament': 'bind',
'prerogative': 'immunity',
'profuse': 'numerous',
'progress': 'pace',
'prohibit': 'taboo',
'prominent': 'conspicuous',
'prompt': 'nimble',
'propagate': 'inseminate',
'propitious': 'successful',
'prudence': 'vigilance',
'punish': 'chastise',
'put': 'place',
'quack': 'impostor',
'quaint': 'queer',
'quarantine': 'seclude',
'queer': 'quaint',
'quell': 'subdue',
'question': 'ask',
'quibble': 'equivocate',
'quick': 'fast',
'quickness': 'rapidity',
'race': 'run',
'raid': 'incursion',
'rapidity': 'quickness',
'rebellious': 'restless',
'rebuke': 'censure',
'recalcitrant': 'mutinous',
'recant': 'retract',
'recede': 'go',
'recount': 'say',
'recover': 'redeem',
'rectify': 'amend',
'redeem': 'recover',
'region': 'plan',
'regret': 'remorse',
'rejoicing': 'jubilant',
'relation': 'kindred',
'relinquish': 'waive',
'remnant': 'residue',
'remonstrate': 'dissuade',
'remorse': 'regret',
'remorseless': 'ruthless',
'reply': 'answer',
'rescind': 'annul',
'resentment': 'displeasure',
'reserved': 'taciturn',
'residue': 'remnant',
'resignation': 'humility',
'respect': 'reverence',
'restless': 'rebellious',
'restrain': 'detain',
'result': 'outcome',
'results': 'outcomes',
'retard': 'hamper',
'retract': 'recant',
'revealed': 'shared',
'reverence': 'respect',
'right': 'correct',
'rout': 'vanquish',
'role': 'part',
'rude': 'awkward',
'run': 'race',
'rural': 'rustic',
'rustic': 'rural',
'ruthless': 'remorseless',
'sacred': 'cherish',
'sarcastic': 'derogatory',
'saucy': 'impudent',
'savage': 'wild',
'savory': 'delicious',
'say': 'recount',
'scanty': 'scarce',
'scarce': 'scanty',
'scared': 'panicked',
'scheme': 'system',
'scorn': 'contempt',
'scrutinize': 'look',
'seclude': 'quarantine',
'servile': 'slavish',
'shared': 'revealed',
'shabby': 'miserable',
'shadowy': 'dark',
'shout': 'yell',
'show': 'display',
'simple': 'classic',
'shift': 'move',
'shifted': 'moved',
'slack': 'lax',
'slander': 'defame',
'slavish': 'servile',
'slay': 'kill',
'slice': 'cut',
'slow': 'unhurried',
'sluggish': 'languid',
'snapshot': 'glimpse',
'soar': 'fly',
'solicit': 'entreat',
'sound': 'lucid',
'soewd': 'planted',
'sparkling': 'bright',
'spend': 'defray',
'spirited': 'vivacious',
'sporadic': 'intermittent',
'spurious': 'fake',
'squalid': 'dirty',
'stain': 'blemish',
'start': 'begin',
'started': 'begun',
'startled': 'frightened',
'stature': 'merit',
'steep': 'course',
'stop': 'end',
'story': 'tale',
'stout': 'fat',
'strange': 'eccentric',
'stranger': 'immigrant',
'strife': 'feud',
'stubborn': 'tenacious',
'stuff': 'glut',
'stumble': 'falter',
'stupor': 'lethargy',
'subdue': 'quell',
'sublime': 'magnificent',
'submerge': 'immerse',
'subsequent': 'consequent',
'substantial': 'considerable',
'subterfuge': 'deceit',
'subvert': 'demolish',
'successful': 'propitious',
'succinct': 'concise',
'summit': 'zenith',
'superficial': 'partial',
'supremacy': 'cominion',
'sycophant': 'parasite',
'sympathy': 'tenderness',
'taboo': 'prohibit',
'taciturn': 'reserved',
'tale': 'story',
'tame': 'compliant',
'tangled': 'intricate',
'tasteless': 'insipid',
'tell': 'disclose',
'temporal': 'transient',
'tenderness': 'sympathy',
'tenement': 'apartment',
'terminate': 'cease',
'terse': 'incisive',
'thick': 'chunky',
'think': 'consider',
'thought': 'idea',
'thoughtful': 'judicious',
'thrifty': 'frugal',
'throng': 'assembly',
'timid': 'diffident',
'tired': 'jaded',
'tomorrow': 'the day after today',
'trace': 'track',
'traced': 'tracked',
'track': 'trace',
'traced': 'tracked',
'tranquil': 'quiet',
'transient': 'temporal',
'transparent': 'diaphanous',
'tremble': 'vibrate',
'trenchant': 'assertive',
'trickery': 'wile',
'trifling': 'trivial',
'trim': 'neat',
'trivial': 'trifling',
'trouble': 'distress',
'tyro': 'novice',
'ugly': 'horrible',
'umbrage': 'chagrin',
'unavoidable': 'inevitable',
'uncertain': 'equivocal',
'unconquerable': 'invincible',
'uncontrite': 'impenitent',
'uncouth': 'boorish',
'undertake': 'endeavour',
'undisguised': 'ingenuous',
'unfortunate': 'hapless',
'unhurried': 'slow',
'urchin': 'foundling',
'urge': 'incite',
'utterly': 'completely',
'vagrant': 'wander',
'vain': 'abortive',
'valor': 'bravery',
'valid': 'correct',
'validate': 'verify',
'vanquish': 'rout',
'venerable': 'esteemed',
'vengefulness': 'malice',
'venom': 'poison',
'veteran': 'ingenious',
'vibrate': 'tremble',
'vicious': 'wicked',
'vigilance': 'prudence',
'vilify': 'malign',
'violate': 'infringe',
'violent': 'tumultuous',
'virtue': 'ethic',
'virtuous': 'chaste',
'vivacious': 'spirited',
'vivid': 'eloquent',
'vouch': 'confirm',
'vulgar': 'base',
'waive': 'relinquish',
'wan': 'pale',
'ways': 'methods',
'wander': 'vagrant',
'wane': 'decline',
'weak': 'fragile',
'wearisome': 'tedious',
'wed': 'marry',
'weep': 'cry',
'wellness': 'fitness',
'wield': 'exert',
'wild': 'savage',
'wile': 'trickery',
'wilt': 'wither',
'wither': 'wilt',
'wrathful': 'angry',
'yearn': 'languish',
'yell': 'shout',
'yield': 'concede',
'yoke': 'connect',
'young': 'juvenile',
'zeal': 'eagerness',
'zenith': 'summit',
'zig-zag': 'oblique'}

In [4]:
sample_text = "The concept of leadership can be traced back to the very beginning of time in the garden of Eden where the first man, Adam had dominion over every living thing in the garden. Since then, kings have begun to appear. In every gathering, a king was appointed at every rally. Leaders were usually appointed from birth or from God’s instruction. In the Middle Ages, leaders were authoritarian, intimidating, fearful, and rare. As a result of the Renaissance, leadership has moved from the kings alone to parliament which consisted of simple men. This left a question, where did the authority come from? Isn't it from God? Jean-Jacques in 1762 made us know Authority comes from people who give up their personal freedom for their safety and security. Anxiety, robbery, murder was the order of the day for leadership in the Middle Ages. Fear and obedience were planted in the hearts of people between 500 AD and 1300 AD.19th and 20th Century Leadership: The Beginning of the Testimony of the Great Leaders That Emerged Leaders such as Abraham Lincoln of the United States, Giuseppe Mazzini of Italy, Kaiser Wilhelm II of Germany. Scholars have begun to pay more attention to leadership as theories were forming. However, in the 21st century, the emergence of information and communication Technology further evolved the concept of leadership as it provided a platform for greater inclusion in leadership, decision making and planning through globalization."
sample_text_2 = '''Today Apple shared a new report that offers a snapshot of the ways Apple products are empowering people to be at the centre of their health, and acting as an intelligent guardian for their health and safety. Users, developers, medical institutions, and health organisations around the world are using Apple devices, features, and APIs to break down barriers between people and their health information, all while keeping privacy in mind.
Apple’s efforts to 
advance health primarily fall into two categories, which are detailed in two corresponding sections of the report. The first section describes Apple’s focus on personal health and fitness features on Apple Watch and iPhone that offer actionable, science-based insights and help protect users’ health and safety. The second section shares Apple’s work with the medical community to support research and care. Both sections — along with an Extensions and Spotlights section at the report’s end — include a variety of examples of third-party developers, health institutions, and organisations innovating with Apple technology.
“We believe passionately that technology can play a role in improving health outcomes and encouraging people to live a healthier day, and we are excited about the many ways users are benefiting from our health and fitness features, and by the ways third-party developers, institutions, and organisations are using Apple technology to advance health and science,” said Jeff Williams, Apple’s chief operating officer. “Our vision for the future is to continue to create science-based technology that equips people with even more information and acts as an intelligent guardian for their health, so they’re no longer passengers on their own health journey. Instead, we want people to be firmly in the driver’s seat with meaningful, actionable insights.”
'''

In [5]:
class Summarizer():
    '''
    Class containing functions to clean, format and summarize text
    Parameters:
    > self : text, type - string
    > self : churn_level float specifying percentage of original content to capture
    '''


    def __init__ (self, text, churn_level):
        self.text = text
        self.churn_level = float(churn_level)
        self.text_topic = '' # List that will contain string topics.

    
    def word_sentence_tokenizer(self):
        """ 
        This function breaks text into word and sentence tokens   
        
        Parameters:

        > self : text, type -string
        > self : churn_level float specifying percentage of original content to capture

        return:

        (sent_tokens, word_tokens) : Tuple containing sentences (sentence tokens) and
        texts (text tokens) contained in text provided
        """

        sent_tokens = sent_tokenize(self.text, 'english')
        word_tokens = word_tokenize(self.text, 'english')
        return(sent_tokens, word_tokens)

    
    def word_count_vec(self, word_tokens):
        '''
         This function produces a dictionary containing the normalized scores of each word tokens in a list
         
         Parameters:
         
         > word_tokens = [] # List of words
         
         return:

         word_frequency_scores : Dictionary of word tokens and their normalized scores

        '''
        clean_words = []
        word_frequency_scores = {}

        # Looping through to calculate word frequencies
        for word in word_tokens:
            if word.strip().lower() not in stop_words:
                if word not in punctuation:
                    clean_words.append(word)
                    if word not in word_frequency_scores:
                        word_frequency_scores[word] = 1
                    else:
                        word_frequency_scores[word] += 1
        
        # Looping through to normalize word_frequency_scores using linear / minmax scaler
        max_frequency = max(word_frequency_scores.values())
        min_frequency = min(word_frequency_scores.values())
        for word in word_frequency_scores.keys():
            word_frequency_scores[word] = (word_frequency_scores[word] - min_frequency) / (max_frequency - min_frequency)

        topic = max(word_frequency_scores, key=word_frequency_scores.get)
        self.text_topic += topic
        return(word_frequency_scores)
    

    def sentence_scoring(self, sentence_tokens, word_frequency_scores):
        '''
        This function calculates scores for each sentence and returns a dictionary containing sentence, score and order.
        
        Parameters:

        > sentence_tokens: List containing sentence tokens
        > word_frequency_scores: Dictionary containing word tokens and their (normalized) scores

        return:

        sentence_scores : Dictionary of sentences and their scores.

        '''
        sentence_scores = {}
        for sentence in sentence_tokens:
            for word in word_tokenize(sentence, 'english'):
                if word.lower() in word_frequency_scores.keys():
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequency_scores[word.lower()]
                    else:
                        sentence_scores[sentence] += word_frequency_scores[word.lower()]
        return(sentence_scores)

        
    def summary_sorting(self, sentence_scores):
        '''
        This function selects the top n sentences based on the sentence scores
        then organizes the final sentence in asccending order of how they appeared in original text

        Parameters:

        self:churn_level - percentage of original content to capture
        sentence_scores - Dictionary containing sentences and their scores.

        return:

        final_summary : String of final / formatted summary output.
        '''

        order_sorted_sentences = []
        score_sorted_sentences = []
        sentence_score_order_tuples = []

        # multiplying churn level by number of sentences then converting to integer 
        top_n_sentences = int(self.churn_level * len(sentence_scores.keys()))

        order = 1
        # sort all sentences in descending order of their sentence_score values
        for sentence, score in sentence_scores.items():
            sentence_score_order_tuples.append((sentence, score, order))
            order += 1
        score_sorted_sentences = sorted(sentence_score_order_tuples, key=lambda tup: tup[1], reverse=True)
        # Slicing from first to top_n_sentences and appending result to produce final summary.
        top_n_slice = score_sorted_sentences[0:top_n_sentences]
        order_sorted_sentences = sorted(top_n_slice, key=lambda tup: tup[2], reverse=False)
        final_summary_list = [sentence[0] for sentence in order_sorted_sentences]
        final_sorted_summary_string = ' '.join(final_summary_list)
        return(final_sorted_summary_string)


def extract_txt(document):
    """
    Function to extract text from .txt file extension document

    Parameters:
    
    > Document with file extension .txt
    
    return:
    
    full_text_string : String of text contained in the .txt document provided
    """
    with open(document) as text:
        full_text_string = text.read().replace("\n", '')
        return(full_text_string)
    

def string_synonym_swap(text):
    """
    This function converts strings to their synonyms    
    It also returns text containing CAPITAL letters or ending with 's', 'ing' , 'ed' as they are,
    including some specified texts whose synonyms are relative to how they appear in sentences.

    Parameters:
    
    > text_list : Strings to be converted to synonyms

    return:

    > test_synonyms : Synonym converted string of text provided. 
    """
    synonyms = [] # final list of synonyms with first index
    text_list = text.split()
    # text_list = word_tokenize(text)
    
    for text in text_list:
        text = text.replace('”', '').replace('“', '') # This helps to conquire the problem behing reported speech "This is reported speech content being altered."
        try:
            if text.islower() and len(text) >= 3:
                synonyms.append(words_synonyms[text])
            elif text in stop_words or text in punctuation or len(text) <3:
                synonyms.append(words_synonyms[text])
            else:
                synonyms.append(text)
        except Exception:
            synonyms.append(text)
        
        # Loops through each token, checks if the token is a punctuation. if it is not a punctuation, it appends the token with a space before to the string-text body
        # if the token is a punctuation, it appens the token to the text body without a space before.
        # 'what is that?' will appear as 'what is that ? ' if this for loop didn't exist.
        string = ''
        for token in synonyms:
            if token not in punctuation:
                string += ' '+token
            else:
                string += token 
    return(string.strip())

In [6]:
sum_text = sample_text_2

a = Summarizer(sum_text, 0.6)
words = a.word_sentence_tokenizer()
words_ = words[1]
sentence = words[0]
token_scores = a.word_count_vec(words_)
test = a.sentence_scoring(sentence, token_scores)

fine = a.summary_sorting(test)
final = string_synonym_swap(fine)

# print(len(fine))
# print(len(sample_text_2))
print(f'The topic of this body of text is "{a.text_topic}".', '\n')
print('Original text')
print(sum_text)
print('\n Length of original text: ', len(sum_text))
print('\n Proper summay \n')
print(final)
print('length of summary: ', len(final))

# how to print the topic
# print(a.text_topic)#####################################################

The topic of this body of text is "health". 

Original text
Today Apple shared a new report that offers a snapshot of the ways Apple products are empowering people to be at the centre of their health, and acting as an intelligent guardian for their health and safety. Users, developers, medical institutions, and health organisations around the world are using Apple devices, features, and APIs to break down barriers between people and their health information, all while keeping privacy in mind.
Apple’s efforts to 
advance health primarily fall into two categories, which are detailed in two corresponding sections of the report. The first section describes Apple’s focus on personal health and fitness features on Apple Watch and iPhone that offer actionable, science-based insights and help protect users’ health and safety. The second section shares Apple’s work with the medical community to support research and care. Both sections — along with an Extensions and Spotlights section at the rep

In [18]:
hey = 'today is a great day just like tomorrow wellness fitness equips novels core'
f = string_synonym_swap(hey)
print('Original text\n',hey)
print('Summary\n',f)

Original text
 today is a great day just like tomorrow wellness fitness equips novels core
Summary
 today is a noteworthy day just like the day after today fitness wellness equips novels middle
